In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pickle
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import re

# Setup Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open Facebook login page
driver.get("https://www.cftc.gov/dea/futures/financial_lf.htm")
# input('enter')
pre_element = driver.find_element("tag name", "pre")
pre_text = pre_element.text

print(pre_text)
driver.quit()




 
Traders in Financial Futures - Futures Only Positions as of November 19, 2024                                                                              
-----------------------------------------------------------------------------------------------------------------------------------------------------------
              Dealer            :           Asset Manager/       :            Leveraged           :              Other             :     Nonreportable    :
           Intermediary         :           Institutional        :              Funds             :           Reportables          :       Positions      :
    Long  :   Short  : Spreading:    Long  :   Short  : Spreading:    Long  :   Short  : Spreading:    Long  :   Short  : Spreading:    Long  :   Short   :
-----------------------------------------------------------------------------------------------------------------------------------------------------------
CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE   (CONTRACTS OF 

In [2]:
def is_integer(value):
    try:
        int(value.replace(',', ''))  # Remove commas before checking
        return True
    except ValueError:
        return False

result_list = []  # Initialize the list to store results

lines = pre_text.split("\n")

# Find all lines containing the phrase 'Traders in Financial Futures'
phrase = "Traders in Financial Futures"
matching_lines = []

tickers_line=[]
for idx, line in enumerate(lines):
    if phrase in line:
        matching_lines.append((idx, line))

# Output the results
if matching_lines:
    for line_idx, line in matching_lines:
        # print(f"Line {line_idx}: {line}")
        tickers_line.append(line_idx)
else:
    print(f"The phrase '{phrase}' was not found in the text.")
    

for j in range(len(tickers_line)-1):
    try:
        ticker = {}
        i = tickers_line[j] - 1
        # print(i)
        
        ticker_name = lines[7 + i].split("-")[0]
        line_3 = lines[3 + i].split(":")
        line_4 = lines[4 + i].split(":")
        
        # Clean the column names by stripping spaces from the split parts
        columns_3 = [col.strip() for col in line_3 if col.strip()]  # Remove empty values and strip spaces
        columns_4 = [col.strip() for col in line_4 if col.strip()]
        combined_columns = [f"{col3.lower()}-{col4.lower()}" for col3, col4 in zip(columns_3, columns_4)]
        
        line_5 = lines[5 + i].split(':')
        columns_pos = [col.strip() for col in line_5 if col.strip()]
        
        line_10 = lines[10 + i].split()
        num_positions = [col.strip() for col in line_10 if col.strip()]
        
        line_13 = lines[13 + i].split()
        change_from_last_week = [col.strip() for col in line_13 if col.strip()]
        
        line_19 = lines[19 + i].split()
        number_of_trades = [col.strip() for col in line_19 if col.strip()]
        
        line_12 = lines[12 + i].split()
        total_change = line_12[-1]
        
        numbers = re.findall(r'\d+', lines[7 + i].replace(',', ''))
        num_contracts = [int(num) for num in numbers][0]
        
        lines_8 = lines[8 + i].split()
        
        changes_from=lines[13].split()
        
        # Filter integers from the list
        open_interest = [item for item in lines_8 if is_integer(item)][0]
        
        result = {}
        pos_index = 0

        for i, col in enumerate(combined_columns):
            if i < 4:  # First 4 combined_columns
                result[col] = {
                    columns_pos[pos_index]: num_positions[pos_index],
                    columns_pos[pos_index + 1]: num_positions[pos_index + 1],
                    columns_pos[pos_index + 2]: num_positions[pos_index + 2],
                    'long_changes': changes_from[pos_index],
                    'short_changes':changes_from[pos_index+1],
                    'spreading_changes':changes_from[pos_index+2]
                }
                pos_index += 3
            else:  # Last combined_column
                result[col] = {
                    columns_pos[pos_index]: num_positions[pos_index],
                    columns_pos[pos_index + 1]: num_positions[pos_index + 1],
                    'long_changes': changes_from[pos_index],
                    'short_changes':changes_from[pos_index+1]
                }
                pos_index += 2
        
        # Adding additional information to result
        result['open_interest'] = open_interest
        result['total_change'] = total_change
        result['number_of_contracts'] = num_contracts
        # result['long_changes']=long_changes
        # result['short_changes']=short_changes
        # result['spread_changes']=spreading_changes
        # Appending the result to the result list
        ticker[ticker_name] = result
        result_list.append(ticker)
    
    except Exception as e:
        print(f"Error occurred at index {j}: {e}")
        continue  # Continue to the next iteration if an error happens

Error occurred at index 45: list index out of range


In [4]:
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Tickers Dashboard</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }
        .container {
            padding: 20px;
        }
        .ticker-list {
            display: flex;
            flex-wrap: wrap;
            gap: 10px;
            margin-bottom: 20px;
        }
        .ticker-list button {
            padding: 10px 20px;
            background-color: #007BFF;
            color: white;
            border: none;
            cursor: pointer;
            border-radius: 5px;
        }
        .ticker-list button:hover {
            background-color: #0056b3;
        }
        .details {
            border: 1px solid #ddd;
            padding: 20px;
            border-radius: 5px;
            background-color: #f8f9fa;
        }
        .details h3 {
            margin-top: 0;
        }
        .details table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 10px;
        }
        .details table th,
        .details table td {
            padding: 10px;
            text-align: left;
        }
        .details table th {
            background-color: #007BFF;
            color: white;
        }
        .long, .long-changes {
            color: green;
            font-weight: bold;
        }
        .short, .short-changes {
            color: red;
            font-weight: bold;
        }
        .spreading, .spreading-changes {
            color: black;
            font-weight: bold;
        }
        .arrow {
            font-weight: bold;
            margin-left: 5px;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="ticker-list">
            <h2>Tickers</h2>
"""

# Add tickers as horizontally aligned buttons
for item in result_list:
    for ticker in item.keys():
        html_content += f'<button onclick="showDetails(\'{ticker}\')">{ticker}</button>\n'

# Add JavaScript and details section
html_content += """
        </div>
        <div class="details" id="details">
            <h3>Select a ticker to view details</h3>
        </div>
    </div>
    <script>
        const data = """ + str(result_list).replace("'", '"') + """;

        function formatChange(value) {
            if (!value || value === '-') {
                return `<span class="arrow" style="color: gray;">-</span>`;
            }
            const numValue = parseInt(value.replace(/,/g, ''), 10); // Remove commas for proper parsing
            if (numValue > 0) {
                return `<span class="arrow" style="color: green;">${value} ⬆</span>`;
            } else if (numValue < 0) {
                return `<span class="arrow" style="color: red;">${value} ⬇</span>`;
            } else {
                return `<span class="arrow" style="color: gray;">${value} ↔</span>`;
            }
        }

        function showDetails(ticker) {
            const detailsDiv = document.getElementById("details");
            const tickerData = data.find(item => ticker in item)[ticker];

            let html = `<h3>Details for ${ticker}</h3>`;
            html += `<table>
                        <thead>
                            <tr>
                                <th>Category</th>
                                <th>Long</th>
                                <th>Long Changes</th>
                                <th>Short</th>
                                <th>Short Changes</th>
                                <th>Spreading</th>
                                <th>Spreading Changes</th>
                            </tr>
                        </thead>
                        <tbody>`;
            
            for (const [category, values] of Object.entries(tickerData)) {
                if (typeof values === "object") {
                    html += `<tr>
                                <td>${category}</td>
                                <td class="long">${values.Long || '-'}</td>
                                <td class="long-changes">${formatChange(values.long_changes)}</td>
                                <td class="short">${values.Short || '-'}</td>
                                <td class="short-changes">${formatChange(values.short_changes)}</td>
                                <td class="spreading">${values.Spreading || '-'}</td>
                                <td class="spreading-changes">${formatChange(values.spreading_changes)}</td>
                            </tr>`;
                }
            }

            // Adding remaining fields
            html += `</tbody></table>`;
            html += `<h4>Other Details</h4>`;
            html += `<ul>`;
            for (const [key, value] of Object.entries(tickerData)) {
                if (typeof value !== "object") {
                    html += `<li><strong>${key.replace(/_/g, ' ')}:</strong> ${value}</li>`;
                }
            }
            html += `</ul>`;
            detailsDiv.innerHTML = html;
        }
    </script>
</body>
</html>
"""

# Save the HTML file
with open("tickers_dashboard.html", "w") as file:
    file.write(html_content)

print("HTML file 'tickers_dashboard.html' has been updated.")


HTML file 'tickers_dashboard.html' has been updated.
